In [ ]:
#subtype part
import pandas as pd
import numpy as np

#ste código está a fazer a limpeza de dados, treinamento de modelos para fazer a avaliação de importância de features

df = pd.read_csv('dataset_preprocessed.csv', header=0, index_col=0)
df_clinical_patient_cols = ['Lymph nodes examined positive', 'Nottingham prognostic index','Cellularity', 'Chemotherapy', 'Cohort', 'ER status measured by IHC','HER2 status measured by SNP6', 'Hormone Therapy',
       'Inferred Menopausal State', 'Sex', 'Integrative Cluster',
       'Age at Diagnosis', 'Overall Survival (Months)',
       'Overall Survival Status', 'Pam50 + Claudin-low subtype',
       '3-Gene classifier subtype', "Patient's Vital Status",
       'Primary Tumor Laterality', 'Radio Therapy',
       'Tumor Other Histologic Subtype', 'Type of Breast Surgery',
       'Relapse Free Status', 'Relapse Free Status (Months)']
df_clinical_feature_of_interest = "3-Gene classifier subtype"
df_clinical_patient_cols.remove('3-Gene classifier subtype')
df.drop(columns=df_clinical_patient_cols, inplace=True)


# apagar os na's da feature de interesse, de cada linha
df = df[~df[df_clinical_feature_of_interest].isna()]

from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

print("Reading X and y")
y = df[df_clinical_feature_of_interest].copy()
X = df[[x for x in df.columns if x != df_clinical_feature_of_interest]]
print(X.columns)

print("Value counts of column of interest")
print(y.value_counts())

print("Droping columns")
columns_with_missing_values = X.columns[X.isna().any(axis=0)]
X.drop(columns=columns_with_missing_values, inplace=True)

print("Training classifier")
# classifier = svm.SVC(kernel='linear', C=1, random_state=42)
# classifier = LogisticRegression()
# classifier = DecisionTreeClassifier()
classifier = RandomForestClassifier(n_estimators=111, n_jobs=16)
# classifier = MLPClassifier(hidden_layer_sizes=(200, 100, 100, 100, 100))
# classifier = KNeighborsClassifier(n_neighbors=5)
scores = cross_val_score(classifier, X, y, cv=5)

print(scores)
np.mean(scores)

from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=11, n_jobs=16)
classifier.fit(X,y)
gene_importance = pd.Series(classifier.feature_importances_, index=X.columns)
gene_importance = gene_importance.sort_values(ascending=False)
gene_importance.to_excel('gene_importance26.xlsx')
gene_importance[0:1400].sum()
print("Sum of top 1400 features:", gene_importance[0:1400].sum())



In [ ]:
#has cancer part
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

# gene importance do dataset de expressao genetica de com cancro e sem cancro
# Leitura do novo dataset
df = pd.read_csv('final_nn.csv', sep=',')

# Definir a característica de interesse
df_clinical_feature_of_interest = "hasCancer"

# Pegar todas as colunas e remover a característica de interesse
df_clinical_patient_cols = list(df.columns)
if df_clinical_feature_of_interest in df_clinical_patient_cols:
    df_clinical_patient_cols.remove(df_clinical_feature_of_interest)

# Remover linhas com NaN na característica de interesse
df = df.dropna(subset=[df_clinical_feature_of_interest])

# Definir X e y para o treinamento do modelo
print("Reading X and y")
y = df[df_clinical_feature_of_interest].copy()
X = df.loc[:, df.columns != df_clinical_feature_of_interest].copy()

# Remova colunas com dados não numéricos
X = X.select_dtypes(include=[np.number])

print("Value counts of column of interest")
print(y.value_counts())

# Dropar colunas com valores NaN
print("Dropping columns")
columns_with_missing_values = X.columns[X.isna().any(axis=0)]
X = X.drop(columns=columns_with_missing_values)

# Treinamento do classificador
print("Training classifier")
classifier = RandomForestClassifier(n_estimators=111, n_jobs=16)
try:
    scores = cross_val_score(classifier, X, y, cv=5)
    print("Cross-validation scores:", scores)
    print("Mean score:", np.mean(scores))
except Exception as e:
    print(f"An error occurred: {e}")

# Treinamento do modelo final e extração de importâncias de característica
classifier = RandomForestClassifier(n_estimators=11, n_jobs=16)
classifier.fit(X, y)
gene_importance = pd.Series(classifier.feature_importances_, index=X.columns)
gene_importance = gene_importance.sort_values(ascending=False)
gene_importance.to_excel('gene_importance3.xlsx')
print("Sum of top 1500 features:", gene_importance[0:1500].sum())
